## RFSoC 4x2 Clocking

In [1]:
import xrfclk

In [2]:
# Load the default PYNQ configuration
xrfclk.set_ref_clks(lmk_freq = 245.76, lmx_freq = 491.52) # both PLLs lock

In [3]:
xrfclk.lmk_devices[0]['spi_device']  # note that the LMK chip is 0.0

PosixPath('/dev/spidev0.0')

## RFSoC 4x2 SPI to LMK

You may need to install spidev with `pip install spidev`

In [4]:
import spidev

In [5]:
spi = spidev.SpiDev(0, 0)  # the LMK chip is 0, 0

In [6]:
def lmkTransfer(spi, address, data=0x00, read=True):
    '''
    Do LMK04828 SPI transfer, either read or write.
    What follows is low-level implimentation details:
    Either transfer sends 3 bytes.
    The only difference between read and write
    is that a read ignores the sent data and has a meaningful return.
    See LMK04828 datasheet at https://www.ti.com/lit/ds/symlink/lmk04828.pdf
    p25 has a diagram of the SPI timing, describing the 3 bytes,
        where the read/write bit goes R=1 for read, 0 for write,
        how the address gets split up, and
        1st byte:   R   0   0  A12 A11 A10 A9  A8
        2nd byte:  A7  A6  A5  A4  A3  A2  A1  A0
        3rd byte:  D7  D6  D5  D4  D3  D2  D1  D0
    '''
    read_bit = 0x80 if read else 0x00  # high bit 1 for read, 0 for write
    address_high = (address>>8)&(0x1f)  # high 5 bits
    address_low = address&(0xff)  # low 8 bits
    to_send = [read_bit|address_high, address_low, data] # 3 bytes to send
    readback = spi.xfer(to_send) # a 3-item list, each one byte
    # the first two returned items should be 0. The last is the data in read mode
    #print('address:', hex(address), 'data:', hex(data), 
    #      'read' if read else 'write', 'result:', readback) 
    return readback[2]

def lmkWrite(spi, address, data):
    lmkTransfer(spi, address, data, read=False)  # readback is always 0

def lmkRead(spi, address, data=0x00):
    '''
    On the RFSoC 4x2 board, the STATUS_LD2 pin, attached to wire LMK_LD2
    is connected to both the "PLL2 locked" LED and the SPI readback"
    
    See the bottom of the CLOCK II page, p16 of the schematic:
    https://www.realdigital.org/downloads/3ae3a2552d7da46e9041196c654cd63d.pdf
    
    This pin is, by default, configured to output whether PLL2 is locked, 
    but the can be reconfigured to 18 different functions, 
    including "SPI readback".
    See PLL2_LD_MUX on p97 of the LMK04828 datasheet at:
    https://www.ti.com/lit/ds/symlink/lmk04828.pdf
    
    If this function always returns 0xff or 0x00,
        the PLL2_LD pin is not configured for "SPI readback".
    Instead, in this configuration, 0xff means PLL2 locked 
        and the PLL2 LED is on, whereas and 0x00 means it's not.
    '''
    return lmkTransfer(spi, address, data, read=True)

def lmkSPIreadbackMode(spi):
    '''
    Put the PLL2_LD pin into "SPI readback" and "Output (push-pull)" mode.
    A side effect is that the PLL2 LED on the board will be mostly off
    except for flashes that are too brief to see when you do SPI reads.
    '''
    lmkWrite(spi, address=0x16E, data=0b00111011)

def lmkPLL2lockedLEDmode(spi):
    'Put the PLL2_LD pin (back) into the "PLL2 DLD" and "Output (push-pull)" mode'
    lmkWrite(spi, address=0x16E, data=0b00010011)

In [7]:
lmkSPIreadbackMode(spi)  # needed to read back from the PLL2 LED pin

In [8]:
# Addresses are listed with a table starting on p57 and details on p61

In [9]:
'{:08b}'.format(lmkRead(spi, address = 0x000))   # SPI modes. 
# We're only ever going to see 4-wire SPI mode because we need to read out through LD2

'00010000'

In [10]:
printClockInfo'{:08b}'.format(lmkRead(spi, address = 0x002))   # POWERDOWN if you write a 0x01 to it.

'00000000'

In [11]:
lmkRead(spi, address = 0x003)   # ID_DEVICE_TYPE. Should be 6 for our PLL product device type

6

In [12]:
lmkRead(spi, address = 0x004)  # MSB of the product identifier. LMK should return 208

208

In [13]:
lmkRead(spi, address = 0x005)  # LSB of the product identifier. LMK should return 91

91

In [14]:
lmkRead(spi, address = 0x006)  # IC version identifier. LMK04828 should return 32

32

In [15]:
lmkRead(spi, address = 0x00C)  # MSB of the vendor identifier. Should return 81

81

In [16]:
lmkRead(spi, address = 0x00D)  # LSB of the vendor identifier. LMK04828 should return 4

4

In [17]:
'{:08b}'.format(lmkRead(spi, address = 0x138))   # VCO_MUX, OSCout_MUX, OSCout_FMT

'00000000'

In [18]:
'{:08b}'.format(lmkRead(spi, address = 0x146))   #  CLKin_SEL_POL, CLKin_SEL_MODE, CLKin1_OUT_MUX, CLKin0_OUT_MUX

'00011011'

In [19]:
'{:08b}'.format(lmkRead(spi, address = 0x147))   #  CLKin enable and type controls.

'00011010'

In [20]:
'{:08b}'.format(lmkRead(spi, address = 0x15F))  # PLL1_LD_MUX, PLL1_LD_TYPE  What is LED PLL1 set to otput?

'00001011'

In [21]:
'{:08b}'.format(lmkRead(spi, address = 0x16E))  # PLL2_LD_MUX, PLL2_LD_TYPE  What is LED PLL2 set to otput?

'00111011'

In [22]:
'{:08b}'.format(lmkRead(spi, address = 0x182)) # RB_PLL1_LD_LOST, RB_PLL1_LD, CLR_PLL1_LD_LOST 
# PLL1 lock information. Look at bit 1 for current lock status. 
# Bit 2 for sticky lock loss.

'00000110'

In [23]:
# To use the sticky bit and reset RB_PLL1_LD_LOST, write CLR_PLL1_LD_LOST with 1 and then 0.
lmkWrite(spi, address=0x182, data=0x01)
lmkWrite(spi, address=0x182, data=0x00)
# now read status again
'{:08b}'.format(lmkRead(spi, address = 0x182))

'00000010'

In [24]:
'{:08b}'.format(lmkRead(spi, address = 0x183)) # RB_PLL2_LD_LOST, RB_PLL2_LD, CLR_PLL2_LD_LOST
# PLL2 lock information. Look at bit 1 for current lock status. Bit 2 for sticky lock loss.
# But only if PLL2_LD_MUX is set! Shit.

'00000100'

In [25]:
# To use the sticky bit and reset RB_PLL2_LD_LOST, write CLR_PLL2_LD_LOST with 1 and then 0.
lmkWrite(spi, address=0x183, data=0x01)
lmkWrite(spi, address=0x183, data=0x00)
# now read status again
'{:08b}'.format(lmkRead(spi, address = 0x183))

'00000000'

## RFSoC 4x2 LMK External 10 MHz input (from GPS disciplined oscillator)

In [26]:
'{:08b}'.format(lmkRead(spi, address = 0x184)) # RB_DAC_VALUE(MSB), RB_CLKinX_SEL, RB_CLKinX_LOS
# Are inputs oscillators experiencing a loss-of-signal (LOS) and which is being used (SEL)

'10010000'

In [27]:
def printLMKclockInfo(spi):
    clk_info = '{:08b}'.format(lmkRead(spi, address = 0x184))
    print('CLKin0 (external) selected SEL for PLL1:', clk_info[7-3])
    print('CLKin1 (internal) selected SEL for PLL1:', clk_info[7-4])
    print('CLKin2 (nothing)  selected SEL for PLL1:', clk_info[7-5])
    print('CLKin0 (external) loss-of-signal LOS:', clk_info[7-0])
    print('CLKin1 (internal) loss-of-signal LOS:', clk_info[7-1])

In [28]:
printLMKclockInfo(spi)

CLKin0 (external) selected SEL for PLL1: 0
CLKin1 (internal) selected SEL for PLL1: 1
CLKin2 (nothing)  selected SEL for PLL1: 0
CLKin0 (external) loss-of-signal LOS: 0
CLKin1 (internal) loss-of-signal LOS: 0


In [29]:
'{:08b}'.format(lmkRead(spi, address = 0x188))  # RB_HOLDOVER in bit 4

'00000000'

In [30]:
# select manual CLKin0 (external 10 MHz) input to PLL1
lmkWrite(spi, address = 0x147, data = 0b00001010)
printLMKclockInfo(spi)

CLKin0 (external) selected SEL for PLL1: 1
CLKin1 (internal) selected SEL for PLL1: 0
CLKin2 (nothing)  selected SEL for PLL1: 0
CLKin0 (external) loss-of-signal LOS: 0
CLKin1 (internal) loss-of-signal LOS: 0


In [31]:
# select manual CLKin1 (internal 10 MHz) input to PLL1
lmkWrite(spi, address = 0x147, data = 0b00011010)
printLMKclockInfo(spi)

CLKin0 (external) selected SEL for PLL1: 0
CLKin1 (internal) selected SEL for PLL1: 1
CLKin2 (nothing)  selected SEL for PLL1: 0
CLKin0 (external) loss-of-signal LOS: 0
CLKin1 (internal) loss-of-signal LOS: 0


In [32]:
# allow both  CLKin0 (external 10 MHz) and CLKin1 (internal 10 MHz), 
# but not CLKin2 to be used in auto-switching of CLKin_SEL_MODE.
# note that this is how the PYNQ default file has it already, 
# so this first write is probably redundant
lmkWrite(spi, address = 0x146, data = 0b0011011)
# select auto between CLKin0 (external 10 MHz) and CLKin1 (internal 10 MHz) input to PLL1
lmkWrite(spi, address = 0x147, data = 0b01001010)
# The problem with auto mode is 
# 1. It might not start up on the external 10 MHz if it's present.
# 2. If external 10 MHz is ever lost, it'll just switch to internal and not go back
printLMKclockInfo(spi)

CLKin0 (external) selected SEL for PLL1: 0
CLKin1 (internal) selected SEL for PLL1: 1
CLKin2 (nothing)  selected SEL for PLL1: 0
CLKin0 (external) loss-of-signal LOS: 0
CLKin1 (internal) loss-of-signal LOS: 0


In [33]:
printLMKclockInfo(spi)

CLKin0 (external) selected SEL for PLL1: 0
CLKin1 (internal) selected SEL for PLL1: 1
CLKin2 (nothing)  selected SEL for PLL1: 0
CLKin0 (external) loss-of-signal LOS: 0
CLKin1 (internal) loss-of-signal LOS: 0


In [34]:
lmkPLL2lockedLEDmode(spi)